# Customer Churn Prediction & Analysis Tool

### Importing the libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Reading the Dataset

In [2]:
df = pd.read_csv('../data/churn.csv')

### Frist look at the dataset EDA checklist

In [21]:
#df.head()
#df.shape
#df.info()
df.describe()
#df["Exited"].value_counts(normalize=True)

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


### Splitting the dataset into indipendet variables(X) and dipendent variable (y)

In [17]:
X = df.drop("Exited", axis=1)
y = df["Exited"]

### Encoding Categorical Data: Geography and Gender


In [18]:
categorical_cols = ["Geography", "Gender"]
numerical_cols = X.drop(columns=categorical_cols).columns

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", sparse_output=False), categorical_cols),
        ("num", "passthrough", numerical_cols)
    ]
)
X_clean = preprocessor.fit_transform(X)

In [30]:
# nomi colonne one-hot
ohe_feature_names = preprocessor.named_transformers_["cat"].get_feature_names_out(categorical_cols)

# colonne numeriche
all_feature_names = list(ohe_feature_names) + list(numerical_cols)

In [32]:
X_df = pd.DataFrame(
    X_clean,columns=all_feature_names
)

In [33]:
X_df.head()
X_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Geography_Germany  10000 non-null  object
 1   Geography_Spain    10000 non-null  object
 2   Gender_Male        10000 non-null  object
 3   RowNumber          10000 non-null  object
 4   CustomerId         10000 non-null  object
 5   Surname            10000 non-null  str   
 6   CreditScore        10000 non-null  object
 7   Age                10000 non-null  object
 8   Tenure             10000 non-null  object
 9   Balance            10000 non-null  object
 10  NumOfProducts      10000 non-null  object
 11  HasCrCard          10000 non-null  object
 12  IsActiveMember     10000 non-null  object
 13  EstimatedSalary    10000 non-null  object
dtypes: object(13), str(1)
memory usage: 1.1+ MB
